<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/exercise_trigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quests
- E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model? (COMPLETE!)
- E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see? (COMPLETE!)
- E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
- E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
- E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
- E06: meta-exercise! Think of a fun/interesting exercise and complete it.


In [20]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [21]:
words = open('names.txt', 'r').read().splitlines()

In [22]:
train_idx = int(len(words) * 0.8)
dev_idx = int(len(words) * 0.1)

train_words = words[:train_idx]
dev_words = words[train_idx:train_idx + dev_idx]
test_words = words[train_idx+dev_idx:]

print(f"Train set: {len(train_words)}")
print(f"Dev set: {len(dev_words)}")
print(f"Test set: {len(test_words)}")

Train set: 25626
Dev set: 3203
Test set: 3204


In [23]:
import torch

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

N = torch.zeros((27, 27, 27) , dtype=torch.int32)

for w in train_words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2, ix3] += 1

alpha = 1
N += alpha

In [24]:
P = N.float()
P /= P.sum(2, keepdim=True)

g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix1 = 0
  ix2 = 0
  while True:
    p = P[ix1, ix2]
    ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix3])
    if ix3 == 0:
      break
    ix1, ix2 = ix2, ix3
  print(''.join(out))

junide.
ilyasid.
prelayleinna.
jimritonia.
saree.


In [31]:
def calulate_loss(words):
  g = torch.Generator().manual_seed(2147483647)
  log_likelihood = 0.0
  n = 0

  for w in words:
  # for w in ["kanade"]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      prob = P[ix1, ix2, ix3]
      logprob = torch.log(prob)
      log_likelihood += logprob
      n += 1
      # print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

  print(f'{log_likelihood=}')
  nll = -log_likelihood
  print(f'{nll=}')
  print(f'{nll/n}')
  return "\n"

print(f"Test set {calulate_loss(train_words)}")
print(f"Dev set {calulate_loss(dev_words)}")
print(f"Test set {calulate_loss(test_words)}")

log_likelihood=tensor(-322671.4062)
nll=tensor(322671.4062)
2.053244113922119
Test set 

log_likelihood=tensor(-45202.6094)
nll=tensor(45202.6094)
2.3264338970184326
Dev set 

log_likelihood=tensor(-45653.1797)
nll=tensor(45653.1797)
2.337472677230835
Test set 



# NN with one layer

In [32]:
xs, ys = [], []

for w in train_words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]

    xs.append([ix1, ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.shape[0]
print('number of examples: ', num)

number of examples:  157152


In [33]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True)

In [35]:
import torch.nn.functional as F

# gradient descent
for k in range(1000):

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float()
  xenc = xenc.reshape(xenc.shape[0], -1)

  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)

  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  if k % 10 == 0:
    print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -50* W.grad

2.787389039993286
2.409698486328125
2.328303337097168
2.2924156188964844
2.27254581451416
2.260072946548462
2.251558542251587
2.2453932762145996
2.240731954574585
2.237093687057495
2.234182834625244
2.2318100929260254
2.2298460006713867
2.2281992435455322
2.226803779602051
2.22560977935791
2.224579334259033
2.2236828804016113
2.22289776802063
2.22220516204834
2.2215917110443115
2.2210447788238525
2.220555067062378
2.2201154232025146
2.2197189331054688
2.219360589981079
2.2190349102020264
2.2187392711639404
2.2184698581695557
2.2182233333587646
2.2179975509643555
2.2177908420562744
2.2176003456115723
2.2174248695373535
2.2172632217407227
2.217113733291626
2.216975212097168
2.2168467044830322
2.2167274951934814
2.21661639213562
2.216513156890869
2.216416358947754
2.2163259983062744
2.2162418365478516
2.2161624431610107
2.21608829498291
2.2160181999206543
2.2159523963928223
2.215890884399414
2.215832233428955
2.2157771587371826
2.2157254219055176
2.2156760692596436
2.2156293392181396
2.21

In [36]:
# Helper functions:
def calculate_loss(xs, ys):
  xenc = F.one_hot(xs, num_classes=27).float()
  xenc = xenc.reshape(xenc.shape[0], -1)

  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)

  num = xs.shape[0]
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

  return loss

def one_hot_encode(words):
  xs, ys = [], []

  for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]

      xs.append([ix1, ix2])
      ys.append(ix3)

  xs = torch.tensor(xs)
  ys = torch.tensor(ys)

  return xs, ys

In [37]:
# Dev loss
xs, ys = one_hot_encode(dev_words)
dev_loss = calculate_loss(xs, ys)
print(f"Dev loss: {dev_loss}")

Dev loss: 2.4342474937438965


In [38]:
# Test loss
xs, ys = one_hot_encode(test_words)
test_loss = calculate_loss(xs, ys)
print(f"Dev loss: {test_loss}")

Dev loss: 2.4337499141693115


In [39]:
# sample
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
      xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
      xenc = xenc.reshape(1, -1)
      logits = xenc @ W
      counts = logits.exp()
      p = counts / counts.sum(1, keepdims=True)

      ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
      out.append(itos[ix3])
      if ix3 == 0:
        break
      ix1, ix2 = ix2, ix3
  print(''.join(out))

uunide.
uliagar.
ulen.
amainn.
vorin.
